In [2]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.optim as optim
import torch.nn as nn
import torch

from tqdm import tqdm

def IP_preprcoessing(ip):
    ip = ip.split(".")
    ip = ip[0].zfill(3) + ip[1].zfill(3) + ip[2].zfill(3) + ip[3].zfill(3)
    return [int(ip[i]) for i in range(12)] 
    
def country_preprocessing(country):
    return country_dict[country]


# Data loader 작성
# country의 경우 index값으로 변경, ip의 경우 전처리 진행('.'를 빼고 자리수를 12자리로 맞춤)
class NetFlowData(Dataset):
    def __init__(self, csv_dir = "04_hashed.csv"):
        f = open(csv_dir)
        self.data = f.readlines()[1:]
        f.close()
        self.country_dict = {}

        idx = 0
        
        for i, d in enumerate(tqdm(self.data)):
            d = d.rstrip().split(",")
            c1, c2 = d[-1], d[-2]
            if c1 not in self.country_dict:
                self.country_dict[c1] = idx
                idx += 1
            if c2 not in self.country_dict:
                self.country_dict[c2] = idx
                idx += 1
                
            d[1] = IP_preprcoessing(d[1])
            d[2] = IP_preprcoessing(d[2])
            d[3] = 1 if d[3] == 6 else 0
            
            self.data[i] = [d[1],d[2],d[3],d[4],d[5],d[6],self.country_dict[d[7]],self.country_dict[d[8]]]
            
            
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return (torch.tensor(self.data[idx][0], dtype=torch.float), #src_ip
                    torch.tensor(self.data[idx][1], dtype=torch.float),  # dst_ip
                    torch.tensor([float(self.data[idx][2])]),  # proto
                    torch.tensor([float(self.data[idx][3])]),  # src_port
                    torch.tensor([float(self.data[idx][4])]),  # dst_port
                    torch.tensor([float(self.data[idx][5])]),  # action
                    torch.tensor([self.data[idx][6]], dtype=torch.float),  # src_country
                    torch.tensor([self.data[idx][7]], dtype=torch.float))  # dst_country

data = NetFlowData()

100%|██████████| 5820310/5820310 [01:03<00:00, 91638.02it/s] 


In [2]:
# country_dict = data.country_dict

In [19]:
# hyperparameter 설정
epochs = 10
gpu_num = 1
lr = 0.001
batch_size = 512

# dataloader 설정
dataloader = DataLoader(data, batch_size=batch_size, shuffle=True , num_workers=1)

# AutoEncoder model 설정
class AE(nn.Module):
    def __init__(self, input_size=12):
        super(AE, self).__init__()
        
        
        self.layers = nn.Sequential(
            self.make_linear(input_size, 128),
            self.make_linear(128, 64),
            self.make_linear(64, 4),
            self.make_linear(4, 64),
            self.make_linear(64, 128),
            self.make_linear(64, input_size)
        )
        
    def make_linear(self, input_size, output_size):
        return nn.Sequential(
            nn.Linear(input_size, output_size),
            nn.BatchNorm1d(output_size),
            nn.ReLU()
        )
        
    def forward(self, x):
        return self.layers(x)


device= torch.device('cpu')
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")
    device = torch.device("cuda:{}".format(gpu_num))
print(device)


model = AE().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
losses = []

from tqdm import tqdm
idx = 0


# 학습
for epoch in range(epochs):
    epoch_loss = 0
    for i, x in enumerate(tqdm(dataloader)):
        idx += 1
        
        ip = torch.cat([x[0], x[1]], dim = 0).to(device)
        output = model(ip)
        loss = criterion(output, ip)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        epoch_loss += loss.item()
        if idx % 1000 == 0:
            print(f'{idx} : ,{epoch_loss/1000}')
            losses.append(epoch_loss/1000)
#             print(epoch_loss/100)
            epoch_loss = 0
    

    epoch_losses = losses[(len(dataloader) * epoch) // 1000 : len(dataloader) * (epoch + 1)]
    epoch_losses = sum(epoch_losses) / (len(epoch_losses) + 0.00001)
    print(f"epoch {epoch}  : {epoch_losses}")
    
#     print(f"Training epoch {epoch}... Loss: {}")

  0%|          | 0/11368 [00:00<?, ?it/s]

Using CUDA
cuda:1


  9%|▉         | 1007/11368 [00:22<03:32, 48.77it/s]

1000 : ,11.282505983352662


 18%|█▊        | 2006/11368 [00:42<03:08, 49.79it/s]

...

112000 : ,1.1659174205064773


 94%|█████████▍| 10693/11368 [03:38<00:13, 50.49it/s]

113000 : ,1.1593711977005006


100%|██████████| 11368/11368 [03:51<00:00, 49.01it/s]

epoch 9  : 1.1329815748292424


In [21]:
# 학습된 모델을 encoder 파트와 decoder 파트로 나눔
encoder = model.layers[:4]
decoder = model.layers[4:]


In [22]:

torch.save(encoder, "encoder_latent1_epoch10.pth")
torch.save(decoder, "decoder_latent1_epoch10.pth")
torch.save(model, "model_latent1_epoch10.pth")


In [23]:
import numpy as np


# src_ip와 dst_ip를 encoder를 통해 encoding
# 추가로 encoder의 accuracy도 확인
acc = 0
src_ip = torch.zeros((len(data),4), dtype=torch.float).to(device)
dst_ip = torch.zeros((len(data),4), dtype=torch.float).to(device)

for i, d in enumerate(tqdm(dataloader)):
    
    with torch.no_grad():
        ip = torch.cat([d[0], d[1]], dim = 0).to(device)
        encoder_output = encoder(ip)
        decoder_output = torch.round(decoder(encoder_output))
        acc += torch.sum(decoder_output == ip) / (ip.shape[0] * ip.shape[1])
        try:
            src_ip[batch_size * i : batch_size * (i+1)] = encoder_output[:len(encoder_output)//2]
            dst_ip[batch_size * i : batch_size * (i+1)] = encoder_output[len(encoder_output)//2:]
        except:
            src_ip[batch_size * i :] = encoder_output[:len(encoder_output)//2]
            dst_ip[batch_size * i :] = encoder_output[len(encoder_output)//2:]
        
src_ip = src_ip.detach().cpu().numpy()
dst_ip = dst_ip.detach().cpu().numpy()

print("accuracy : ", acc/len(dataloader))


100%|██████████| 11368/11368 [03:48<00:00, 49.85it/s]

accuracy :  tensor(0.6497, device='cuda:1')


In [264]:
# ip주소를 제외한 나머지 데이터를 불러옴
# 이때 dataloader를 사용하면 속도가 느리기에 일반적으로 data를 불러오는 방식 사용
proto = np.zeros(len(data),dtype=float)
src_port =  np.zeros(len(data),dtype=float)
dst_port =  np.zeros(len(data),dtype=float)
action =  np.zeros(len(data),dtype=float)
src_country =  np.zeros(len(data),dtype=float)
dst_country =  np.zeros(len(data),dtype=float)

data = open("04_hashed.csv").readlines()[1:]


for i, d in enumerate(tqdm(data)):
    d = d.rstrip().split(",")[1:]
    d[3] = float(d[3])
    d[4] = float(d[4])
    proto[i] = float(d[2])
    src_port[i] = 0.0 if d[3] < 1024 else (1.0 if d[3] < 49152 else 2.0)
    dst_port[i] = 0.0 if d[4] < 1024 else (1.0 if d[4] < 49152 else 2.0)
    action[i] = float(d[5])
    src_country[i] = float(country_dict[d[6]])
    dst_country[i] = float(country_dict[d[7]])

    

100%|██████████| 5820310/5820310 [00:14<00:00, 410529.86it/s]


In [305]:
# data 정규화
normal_distribution = lambda data : (data-np.mean(data))/np.std(data)

proto = normal_distribution(proto)
src_port = normal_distribution(src_port)
dst_port = normal_distribution(dst_port)
action = normal_distribution(action)
src_country = normal_distribution(src_country)
dst_country = normal_distribution(dst_country)


In [321]:
# csv 형식으로 전처리 파일 저장
import csv

with open('preprocessed_sample.csv', 'w', newline='') as csvfile:
    fieldnames = ['src_ip1','src_ip2','src_ip3','src_ip4',
                       'dst_ip1','dst_ip2','dst_ip3','dst_ip4',
                       'proto', 'src_port', 'dst_port', 'action',
                       'src_country', 'dst_country']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for i, d in enumerate(tqdm(src_ip)):
 
        if i > 100:
            break
        writer.writerow({
            "src_ip1" : src_ip[i][0],
            "src_ip2" : src_ip[i][1],
            "src_ip3" : src_ip[i][2],
            "src_ip4" : src_ip[i][3],
            "dst_ip1" : dst_ip[i][0],
            "dst_ip2" : dst_ip[i][1],
            "dst_ip3" : dst_ip[i][2],
            "dst_ip4" : dst_ip[i][3],
            "proto" : proto[i],
            "src_port" : src_port[i],
            "dst_port" : dst_port[i],
            "action" : action[i],
            "src_country" : src_country[i],
            "dst_country" : dst_country[i],
         })
    


  0%|          | 101/5820310 [00:00<02:37, 36955.83it/s]
